## Core Prompting Techniques

In this module, we will explore **core prompt engineering techniques** using a local Large Language Model (LLM) through direct API calls. We'll use a locally hosted LM Studio instance running the **OpenChat-3.5-0106** model (a 7B open-source chat model) to experiment with various prompting strategies. All interactions will be done via Python code using the `requests` library to call the model's API.

**What you'll learn in this module:**

- How to connect to a local LLM API (LM Studio's OpenAI-compatible API).
- Crafting prompts with **specific instructions** and using **delimiters** for clarity.
- Using **self-checking mechanisms** to improve the quality of responses.
- Setting **roles and personas** to influence the model's style or expertise.
- Implementing **few-shot prompting** to give the model examples and guide its answers.
- Managing the **context window** to handle longer conversations or documents.

This notebook is designed for software developers with some experience in API calls, and it emphasizes a hands-on approach. You'll find explanations, example code blocks, and exercises to try out each concept. Let's get started!


### 1. Setup

Let's define a helper function to send chat prompts to the model for convenience. This function will call the `/v1/chat/completions` endpoint with the given messages and parameters:


In [ ]:
import json

# Configure the base URL for the LM Studio API
base_url = "http://localhost:1234/v1"  # adjust the port if needed

# Use the model ID from the models list (replace 'openchat_3.5' with the actual ID if different)
MODEL_ID = "openchat_3.5"  # adjust if needed based on output above

def chat_completion(messages, **kwargs):
    """Send a chat completion request to the local LLM and return the assistant's reply."""
    url = f"{base_url}/chat/completions"
    payload = {
        "model": MODEL_ID,
        "messages": messages
    }
    # Add any additional parameters (like temperature, max_tokens) into payload
    payload.update(kwargs)
    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()  # will raise an HTTPError if status >= 400
    except Exception as e:
        print("Error during API call:", e)
        if response is not None:
            print("Response:", response.text)
        return None
    
    data = response.json()
    # The OpenAI-compatible API returns a dict with 'choices', where each choice has a 'message'
    # We'll return the content of the first choice's message (the assistant's reply).
    try:
        return data['choices'][0]['message']['content']
    except (KeyError, IndexError):
        # If the response format is unexpected, print it for debugging
        print("Unexpected response format:", data)
        return None

# Quick test prompt to ensure everything is working:
test_messages = [{"role": "user", "content": "Hello, how are you?"}]
reply = chat_completion(test_messages)
print("Model reply:", reply)


Run the test above. If everything is set up correctly, the model should respond with a greeting or a brief answer. This confirms we can communicate with the LLM. Now we are ready to explore various prompting techniques.

### 2. Specific Instructions and Delimiter Usage

A well-crafted prompt should give the model **clear instructions** on what you want, including the desired format of the answer. Models often follow instructions literally, so being explicit can greatly improve the quality of the output. Additionally, using **delimiters** (such as quotes, triple backticks, or XML-like tags) to separate different parts of your prompt can help avoid confusion and ensure the model treats given text as input data rather than instructions.

**Key ideas:**
- **Tell the model exactly what format or content you expect.** For example, "Answer in one sentence," or "Provide the output as JSON."
- **Use delimiters to isolate data or code.** For example, use triple backticks ``` to enclose a chunk of text or code that the model should consider as a given input or example, not something to act on except as instructed. Delimiters can be quotes, brackets, or any clearly signaled tokens.

Let's illustrate this with an example. We'll ask the model to transform text into a specific format, first **without clear instruction**, and then **with explicit instruction and delimiters**:

**Example 1:** Without specific instructions  
We have a list of words and we want a comma-separated string of those words. If we just ask plainly, the model might include extra text in its answer:


In [ ]:
prompt = "Can you convert the following list of words into a comma-separated string: apple banana cherry"
messages = [{"role": "user", "content": prompt}]
response = chat_completion(messages)
print(response)

*Run the above.* You might get something like: _"Sure! The comma-separated string is: apple, banana, cherry."_ 

The model did the task but also included additional text ("Sure! The comma-separated string is:"). This is because we didn't explicitly tell it *not* to add extra commentary.

**Example 2:** With explicit instruction and delimiters  
Now, let's give a very specific instruction and use delimiters to clearly mark the input list:


In [ ]:
prompt = (
    "You will be given a list of words. Convert it to a single comma-separated string.\n"
    "Respond with *only* the comma-separated string and no extra words.\n\n"
    "List of words:\n```apple banana cherry```"
)
messages = [{"role": "user", "content": prompt}]
response = chat_completion(messages)
print(response)

In this prompt:
- We explicitly tell the model to respond with *only* the comma-separated string.
- We put the list of words inside a triple backtick block ```...``` to clearly delimit the input data from our instruction.

*Run the cell.* The expected output now should be exactly:  
`apple, banana, cherry`  
(with no extra explanation or formatting).

Notice how the use of clear instructions and delimiters guided the model to give the answer in the precise format we wanted.

**Delimiter examples:** You can use different types of delimiters based on context:
- Triple backticks for multi-line text or code (as shown above).
- Quotes for shorter phrases or single-line inputs (e.g., `"apple banana cherry"`).
- XML/HTML-like tags if needed, e.g., `<input>...</input>` to wrap input data, though this is less common.

The goal is to remove ambiguity. By isolating the data and explicitly stating the requirements, you reduce the model's chances of misunderstanding.

### Exercise: Specific Instructions
1. **JSON output exercise:** Ask the model for some structured data in JSON format. For example: *"Provide the weather forecast for today in JSON format with keys 'temperature', 'condition', and 'humidity'."*  
   - First, try without emphasizing "in JSON only" and see if the model adds extra text.
   - Then try again with an explicit instruction like "Output only valid JSON."
   - Use delimiters if you provide any example JSON or to indicate where the answer should focus.
2. **Using different delimiters:** Give the model a piece of text (like a paragraph) and instruct it to do something (summarize or find a word) with that text. Use a delimiter to enclose the text. For example:  
   *"Here is a paragraph enclosed in triple backticks. ```<paragraph>``` Summarize it in one sentence."*  
   See if the model correctly identifies the bounded text and only uses that for the summary.
3. Try formulating a prompt that includes multiple parts, e.g., *"Take the text between <start> and <end> and translate it to French,"* and see if using `<start>` and `<end>` markers helps the model focus on the right text.

By practicing with specific instructions and delimiters, you'll learn how to **minimize miscommunication** with the model and get outputs closer to what you envision.


In [ ]:
### Feel free to work on the exercises in this cell! ###

### 3. Self-Checking Mechanisms

Sometimes the first answer a model gives might not be fully correct or might miss some requirements. One powerful technique is to prompt the model to **check its own output** before finalizing the answer. This leverages the model's reasoning capabilities to catch mistakes or omissions. Essentially, you ask the model to *think twice*.

**How to implement self-checking:**
- You can do this in a **single prompt** by instructing the model to provide an answer and then verify it.
- Or do it in **multiple steps**: first get an answer, then ask the model to critique or improve that answer.

Let's try the single-prompt approach with a simple example. We'll ask a question and tell the model to answer it and then double-check that the answer is correct and complete:

In [ ]:
prompt = (
    "You are a helpful assistant.\n"
    "Answer the user's question, then check if your answer might be incorrect or missing something. "
    "If you find an issue, correct your answer.\n\n"
    "User's question: Calculate the sum of all even numbers from 1 to 20."
)
messages = [{"role": "user", "content": prompt}]
response = chat_completion(messages)
print(response)

In this prompt, we did a few things:
- Set a role ("You are a helpful assistant.") as context.
- Gave a two-part instruction: (1) answer the question, (2) check the answer for errors or omissions and correct if needed.
- Then provided the actual user question.

*Run the cell.* You should see the model's answer and its self-check reasoning. For example, it might output something like:
```
The sum of all even numbers from 1 to 20 is 110.

(Checking: The even numbers from 1 to 20 are 2,4,...,20. 
Their sum is indeed 110. The answer is correct.)
```
The model first gave an answer "110", then followed up with a check and confirmed it. If the model made a mistake initially, the instruction encourages it to catch that and fix it.

> Note: Not all models will perfectly self-correct, especially smaller ones. OpenChat-3.5 (7B) can handle simple checks, but the effectiveness may vary with question complexity. Still, the process of self-checking often improves accuracy.

Another way to use self-check is a multi-step approach:
1. **First call:** Get the answer normally.
2. **Second call:** Provide the answer back to the model and ask it to evaluate or verify it.


In [ ]:
# Step 1: Ask the question normally
question = "What year did the first man land on the moon?"
messages1 = [{"role": "user", "content": question}]
answer = chat_completion(messages1)
print("Initial answer:", answer)

# Step 2: Ask the model to verify the answer
verify_prompt = f"The answer given was: '{answer}'. Is this answer correct? If not, what is the correct answer?"
messages2 = [{"role": "user", "content": verify_prompt}]
verification = chat_completion(messages2)
print("\nSelf-check result:", verification)


In the above:
- The model might first answer "1969" (which is correct for the moon landing year).
- The second prompt explicitly asks the model to reflect on that answer. Ideally, it will confirm "Yes, 1969 is correct." If the first answer was wrong, the model might correct it in the second step.

*Run these steps* to see how the model handles it. This pattern is useful: get an answer, then have the model act as a reviewer of that answer.

### Exercise: Self-Check Your Answers
- Try a complex question with multiple requirements. For example: *"List the first 3 prime numbers and explain why they are prime."* This has two parts: listing and explaining. The model might list but forget to explain or vice versa. Use a self-check prompt to see if it catches a missing part.
- Ask a trivia question or a math problem in the first call. In the second call, ask the model to double-check or provide reasoning. See if it finds the same answer via reasoning or identifies a mistake.
- Experiment with instructing different styles of self-check. For instance, "Provide your answer, then ask yourself if the user could be dissatisfied with anything in it, and improve it."

Using self-checking mechanisms encourages the model to **critique and improve its own responses**, which can lead to more reliable outputs especially for complicated tasks.


In [ ]:
### Feel free to work on the exercises in this cell! ###

### 4. Role & Persona Prompting

LLMs can change their tone, style, or level of expertise based on the role or persona you assign to them. By specifying a role, you can influence how the model responds. This is often done using the **system message** in a chat or by instructing the model to "act as" someone in the prompt.

**Why use roles/personas?**
- To get responses at a certain expertise level (e.g., a senior developer vs. a beginner's explanation).
- To enforce a style or tone (formal, casual, humorous, scientific, etc.).
- To tap into domain-specific knowledge or behavior (e.g., acting as a database assistant might yield more structured responses for SQL questions).

Let's see how to do this. We'll use the system role to define a persona for the assistant, and then ask a question:


In [ ]:
# Define a system message to set the role/persona of the assistant
system_message = {"role": "system", "content": "You are an expert Python developer and database administrator. \
You speak in a concise, technical manner and provide answers with code examples when relevant."}

# Define a user question to ask the assistant
user_message = {"role": "user", "content": "How would you design a database schema for a simple blog platform?"}

messages = [system_message, user_message]
response = chat_completion(messages, temperature=0.5)
print(response)

In this prompt:
- The **system message** establishes that the assistant is an expert in Python and databases, with a concise and technical style.
- The user's question then asks about designing a database schema for a blog.

*Run the cell.* The expected answer would be in a knowledgeable tone. For example, the assistant might outline tables like Users, Posts, Comments with key fields, possibly even providing a short SQL DDL snippet (since we encouraged code examples).

Now, compare that with a different persona. Let's say we want the answer in a more beginner-friendly tone:


In [ ]:
# Change the system persona to a friendly teacher for beginners
system_message = {"role": "system", "content": "You are a friendly instructor who explains database concepts to beginners. \
You use simple language and analogies, and avoid too much technical jargon."}

messages = [system_message, user_message]  # user_message is the same question as before
response = chat_completion(messages, temperature=0.5)
print(response)


*Run the cell.* This time the answer might be more explanatory and simplified, perhaps explaining what a database schema is, analogies (like comparing a table to an Excel sheet, etc.), before describing a possible schema.

You can see how the **role/persona** shaped the response:
- The expert persona gave a concise, technical design.
- The instructor persona gave a gentle, educational explanation.

You can also set personas by instruction in the user prompt (e.g., "Act as a Shakespearean poet and answer..."), but using the system role (if your API supports it, as OpenAI-compatible APIs do) is cleaner and keeps the user query separate.


### Exercise: Role Play with the Model
- Try other roles. For example, set the system message to: "You are a sarcastic assistant." or "You are a motivational coach." and ask any question (even a generic one like "I need advice on learning programming."). See how the tone changes.
- Use a domain-specific persona: "You are an expert cybersecurity analyst." or "You are a medical encyclopedia." and then ask a relevant question in that domain. Observe how the detail and focus of the answer adapt.
- Experiment with multiple traits: e.g., "You are a **friendly and humorous** chatbot that likes to make light jokes." to combine tones, and ask something like "Explain cloud computing." The result should be informative but with a playful twist.

Using roles and personas helps customize the output style and depth, making the assistant's responses more suitable for different audiences or applications.


In [20]:
### Feel free to work on the exercises in this cell! ###

### 5. Few-Shot Prompting

Few-shot prompting is a technique where you provide **examples** of the task you want the model to perform, to guide it towards the desired output. Instead of just giving an instruction, you show the model "here are some inputs and their correct outputs; now do the same for a new input." This is especially useful if the model isn't following instructions well, or if the task has a specific pattern you want to enforce.

There are two common ways to do few-shot prompting:
- **In a single message (for completion-style)**: Provide examples in the text of the prompt and then the new query.
- **As a series of messages (for chat-style)**: Simulate a conversation with user queries and assistant answers as examples, then provide a final user query for the model to answer.

We'll demonstrate the chat-style approach here (since we're using the chat API), which means we'll include example Q&A pairs as part of the conversation history.

**Example:** Let's use few-shot prompting for a translation task. We'll give the model a couple of examples of translating English to French, then ask it to translate a new sentence.


In [21]:
# Few-shot example for translation
system_msg = {"role": "system", "content": "You are a translation assistant that translates English to French."}
# Example 1
example_user1 = {"role": "user", "content": "Translate to French: Hello, how are you?"}
example_assistant1 = {"role": "assistant", "content": "Traduction: Bonjour, comment ça va ?"}
# Example 2
example_user2 = {"role": "user", "content": "Translate to French: Good morning"}
example_assistant2 = {"role": "assistant", "content": "Traduction: Bonjour"}
# Now the real prompt
real_user = {"role": "user", "content": "Translate to French: I have a yellow car"}

messages = [system_msg, example_user1, example_assistant1, example_user2, example_assistant2, real_user]
response = chat_completion(messages, temperature=0.3)
print(response)

 Traduction: J'ai une voiture jaune


Let's break down what's happening:
- The system message sets the context that the assistant is a translation assistant.
- We provided two example interactions:
  - User says "Translate to French: Hello, how are you?" and Assistant responded with "Traduction: Bonjour, comment ça va ?".
  - User says "Translate to French: Good morning" and Assistant responded with "Traduction: Bonjour".
- Now we ask the actual question: "Translate to French: I have a yellow car".

The idea is that the model will recognize the pattern from the examples and follow it. We even included the word "Traduction:" in the responses to show the format we expect.

*Run the cell.* The model should output something like:
```
Traduction: J'ai une voiture jaune
```
which is the translation of "I have a yellow car" in the same format as the examples.

Few-shot examples don't have to use the system/user/assistant roles exactly like above; you could also embed multiple Q&A in one user prompt. But providing them as separate messages as shown often makes it clearer for the chat model (it sees a conversation with resolved answers).

**Example 2 (classification task):** Suppose we want the model to classify sentiments as Positive/Negative. We can give a few examples:


In [22]:
system_msg = {"role": "system", "content": "You are a sentiment analysis bot. Provide only the sentiment (Positive or Negative) for each review."}
# Few-shot examples
ex_user1 = {"role": "user", "content": "Review: \"I absolutely loved the new design, it's fantastic!\""}
ex_assistant1 = {"role": "assistant", "content": "Positive"}
ex_user2 = {"role": "user", "content": "Review: \"This is the worst experience I've ever had with this product.\""}
ex_assistant2 = {"role": "assistant", "content": "Negative"}
# Actual prompt to analyze
real_user = {"role": "user", "content": "Review: \"The product is okay, not great but not bad either.\""}
messages = [system_msg, ex_user1, ex_assistant1, ex_user2, ex_assistant2, real_user]
response = chat_completion(messages, temperature=0)
print(response)

 Neutral


We expect the model to output "Positive" or "Negative" for the new review based on the sentiment. In this case, the review is mixed/okay, but since our examples only had clear positive or negative, the model might choose one (it might say "Negative" because of "not great", or possibly "Positive" because of "not bad either"). The few-shot format, however, ensures it will output just one word (Positive/Negative) and not a full sentence, because we showed that pattern.

*Run the cell* to see the classification. You can adjust the examples or add a third example with a neutral case to guide it if needed.


### Exercise: Few-Shot Prompting
- **Math QA example:** Provide a couple of examples of a math question and answer, then ask a new math question. For instance, show: "Q: 5+3? A: 8" and "Q: 10-4? A: 6" as examples (as separate user and assistant turns), then ask "Q: 7*2? A:". See if the model continues correctly.
- **Style transfer example:** If you want the model to output text in a certain style (say, in the style of a pirate or in Shakespearean English), you can give an example or two of normal text and its "translated" style, then a new line to do similarly.
- **Coding pattern example:** Show the model an input and output example for a coding problem. For instance: Input: an array -> Output: sorted array (just conceptual). Then give a new input to see if it can produce the correct output.

The key with few-shot prompting is to demonstrate the task. This can significantly improve performance on tasks where the model might otherwise be unsure how to format the answer or what exactly you want.

In [23]:
### Feel free to work on the exercises in this cell! ###

### 6. Context Window Management

Language models have a limited **context window**, meaning they can only consider a certain amount of text (tokens) in the prompt (which includes the conversation history and the question) and the output. If you exceed this limit, the model might truncate the input or fail to respond properly. For OpenChat-3.5-0106 (7B, based on Mistral architecture), the context window is around 8192 tokens maximum ([TheBloke/openchat-3.5-0106-GGUF · Hugging Face](https://huggingface.co/TheBloke/openchat-3.5-0106-GGUF#:~:text=Model%20Size%20Context%20Weights%20Serving,ray)), which is quite large, but you still need to manage it carefully for long interactions or documents.

**Why context management matters:**
- In long conversations, old messages might push out of the window. The model will forget anything not included in the prompt.
- If you feed a very large document as context for a single question, it might exceed the limit or use up so many tokens that little room is left for the answer.
- More tokens in the prompt also means slower responses and higher memory use.

**Techniques for managing context:**
- **Summarize or distill** earlier content: Instead of sending the entire chat history as it grows, summarize older parts and include the summary.
- **Retrieve relevant pieces** of context: If you have many documents, pick only the parts relevant to the query (this is the basis of retrieval-augmented generation).
- **Truncate intelligently**: If you must drop content, drop the least relevant or oldest content that is less likely needed.
- **Use the `max_tokens` parameter for output**: Ensure you allocate enough tokens for the answer. For example, if the context is very large, use `max_tokens` to limit the answer length to avoid hitting the limit (though the model usually won't exceed the context limit on its own).

Let's illustrate a simple scenario. We'll simulate a long conversation by repeating a phrase to consume context (not actually 8000 tokens, but enough to illustrate), and see how using a shorter context vs. a longer context affects the model's ability to recall.


In [24]:
# Simulate context buildup
base_question = "User: What is 2+2?\nAssistant: 4\n"
# Repeat the Q&A to simulate a long conversation history
long_history = base_question * 50  # this will create a long string by repetition

# Now ask a new question at the end of this long history
final_query = "User: Can you explain why the sky is blue?\nAssistant:"
prompt = long_history + final_query

# We'll use the completions endpoint here for simplicity since we crafted a text prompt
completion_url = f"{base_url}/completions"
payload = {
    "model": MODEL_ID,
    "prompt": prompt,
    "max_tokens": 100,
    "temperature": 0.5
}
response = requests.post(completion_url, json=payload)
answer = response.json().get('choices', [{}])[0].get('text', '')
print("Assistant answer:", answer[:200], "...")


Assistant answer:  The sky appears blue due to a phenomenon called Rayleigh scattering. Sunlight is composed of many colors, but when it passes through the atmosphere, shorter wavelengths (blue and violet) are scattere ...


In the above:
- We created a `long_history` by repeating a simple Q&A ("What is 2+2? -> 4") many times.
- Then we append a new question "Why is the sky blue?" at the end.
- We call the `/v1/completions` endpoint directly with this prompt (treating it like a plain text completion task). We also set `max_tokens` to 100 to limit the answer length.
- We print the first 200 characters of the answer to see if it answered the last question.

*Run the cell.* The model should ideally answer "why the sky is blue". If the `long_history` was extremely large (approaching the context limit), the model might struggle or truncate. In our simulation, 50 repetitions is not really near 8192 tokens, but it demonstrates how you'd include a history in the prompt.

Now, imagine the history was too long. One solution is to summarize:


In [25]:
# Instead of sending full long_history, we summarize it (here we'll just fake a summary for illustration)
summary = "User asked many simple math questions and assistant answered them correctly. " \
          "So far, the assistant has been helpful and accurate.\n"
# Now only send the summary + last question
prompt_with_summary = summary + final_query

payload["prompt"] = prompt_with_summary
response = requests.post(completion_url, json=payload)
answer = response.json().get('choices', [{}])[0].get('text', '')
print("Assistant answer with summary:", answer[:200], "...")


Assistant answer with summary:  The sky appears blue due to a phenomenon called Rayleigh scattering. When sunlight passes through the atmosphere, it interacts with air molecules and small particles. Shorter wavelengths of light, su ...


Here, we replaced the 50 Q&A repetitions with a short summary of them. This frees up context space for the new answer. The output should still correctly address "why the sky is blue," ideally similarly to before.

In interactive applications (like a chatbot), you might do this dynamically: once the conversation exceeds a certain length, summarize the older parts or drop them.

Another aspect of context management is making sure the model's output doesn't overflow the context. The `max_tokens` parameter we used ensures the model stops after a certain length. If you expect a concise answer, you can set `max_tokens` lower (like 50). If you expect a long answer (like summarizing a long text), you might set it high (maybe 300-500) but need to ensure the input + output stays under the limit.

**Tip:** There's no built-in function in this notebook to count tokens, but a rough approach is to count words or characters as a proxy. For precise control, you would use a tokenizer (like from Hugging Face `transformers` or OpenAI's `tiktoken`) to count tokens of your prompt and adjust if needed. In practice, the API will return an error if the prompt is too large.


## Conclusion

In this hands-on module, we've covered several core prompting techniques:
- Giving **specific instructions** and using **delimiters** to get more precise answers.
- Employing **self-checking** prompts to have the model verify and refine its own outputs.
- Setting **roles/personas** to influence the style and expertise of responses.
- Using **few-shot examples** to guide the model with patterns and improve performance on specific tasks.
- Managing the **context window** by summarizing or truncating content to stay within limits.

By practicing these techniques, you should have a better command over how to get the best out of an LLM like OpenChat-3.5. Prompt engineering is as much an art as a science – it involves experimentation. Feel free to tweak the examples, try your own prompts, and even combine techniques (e.g., few-shot + a specific persona, or self-checking + a specific format).
